```oarsub -I --project data-ocean -p "cpumodel = 'Gold 5218'" -l /core=2,walltime=4:00:00```

## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask

# The workers

In [2]:
ask_cores=2
ask_memory=4.2 #in GB

In [4]:
from dask_jobqueue import OARCluster 
from dask.distributed import Client 
  
cluster = OARCluster(cores=ask_cores,processes=1,name='pangeo',walltime='01:30:00',
                       job_extra=['--project data-ocean', '-p "cpumodel = `Gold 5218`"'],memory=str(ask_memory)+'GB',
                       interface='ib0') 
cluster.scale(1)
c= Client(cluster)
c


Task exception was never retrieved
future: <Task finished coro=<_wrap_awaitable() done, defined at /home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/asyncio/tasks.py:530> exception=FileNotFoundError(2, "No such file or directory: 'oarsub'")>
Traceback (most recent call last):
  File "/home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/asyncio/tasks.py", line 537, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/site-packages/distributed/deploy/spec.py", line 50, in _
    await self.start()
  File "/home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/site-packages/dask_jobqueue/core.py", line 310, in start
    out = await self._submit_job(fn)
  File "/home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/site-packages/dask_jobqueue/oar.py", line 97, in _submit_job
    return self._call(oarsub_command_split)
  File "/home/alberta/miniconda2/envs/perf-pangeo/lib/python3.6/site-packages/dask_jobqu

Client Scheduler: tcp://172.18.19.112:42431 Dashboard: http://172.18.19.112:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)

# The data

In [ ]:
%time ds=xr.open_zarr('/bettik/alberta/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

In [ ]:
ds

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
mean

In [ ]:
%time mean.load()